# Weather Informed Reservoir Operation

California Data Exchange Center: California Department of Water Resources

- URL: https://cdec.water.ca.gov/dynamicapp/wsSensorData 
- Window: Daily and hourly records from 01-01-2008 to 12-31-2022

In [1]:
import warnings
warnings.simplefilter("ignore")

from datetime import datetime, date
import pandas as pd
import numpy as np

import seaborn as sns
sns.set_theme(style="white")
import matplotlib.pyplot as plt
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.graphics.tsaplots import plot_pacf

from sklearn.metrics import mean_squared_error
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.arima.model import ARIMA
from math import sqrt

In [2]:
DATA_PATH = 'Reservoir_Project/Data'

### Loading and selecting CDEC Water data 

In [133]:
# Prepare dataframes for merge

def prepare_station_df(station_data, hourly=False, date_col='OBS DATE'):
    date_type = str(station_data[date_col].dtype)
    value_type = str(station_data['VALUE'].dtype)
    
    # Convert observed dates to shared date format
    if (date_type == "int64" or date_type == "float64"):
      station_data[date_col] = pd.to_datetime(station_data[date_col], format='%Y%m%d')
    else: 
      station_data[date_col] = pd.to_datetime(station_data[date_col]).dt.date
    
    # Remove sensor number prior to grouping
    station_data.drop(columns='SENSOR_NUMBER', inplace=True)

    if (value_type == "object"):        
        station_data['VALUE'].replace(to_replace=r'.RT$', value='NaN', regex=True, inplace=True)
        clean_val = list(map(lambda val: val.replace(',', ''), station_data['VALUE'].values.astype('str')))
        station_data['VALUE'] = list(map(float, clean_val))
    
    # Set date as index
    if (hourly):
      station_data = station_data.groupby(date_col).mean()
    else: 
      station_data.set_index(date_col, inplace=True)

    # Create station df with observed value
    df = station_data[['VALUE']].copy()

    return df

#### Reservoir Inflow

In [134]:
### Folsom Lake (FOL) ###
# Elevation: 466 ft

# Inflow (daily)
FOL_INFLOW_DATA = pd.read_excel(f'{DATA_PATH}/CDEC_Water_Data/FOL/FOL_76.xlsx')
FOL_INFLOW_DATA_DF = prepare_station_df(FOL_INFLOW_DATA)

# Reset value name for df merge
FOL_INFLOW_DATA_DF.rename(columns={"VALUE": "INFLOW"}, inplace=True)

In [135]:
FOL_INFLOW_DATA_DF.head()

,INFLOW
OBS DATE,
2008-01-01,977.0
2008-01-02,1061.0
2008-01-03,1712.0
2008-01-04,NaN
2008-01-05,7072.0


#### Near American River, North Fork

In [136]:
### NF AMERICAN R AT NORTH FORK DAM (NFD) ### 
# Elevation: 715 ft

# FLOW, MEAN DAILY, CFS (daily)
NFD_MEAN_FLOW_DATA = pd.read_excel(f'{DATA_PATH}/CDEC_Water_Data/NFD/NFD_41.xlsx')
NFD_MEAN_FLOW_DF = prepare_station_df(NFD_MEAN_FLOW_DATA)

# Reset value name for df merge
NFD_MEAN_FLOW_DF.rename(columns={"VALUE": "NFD_MEAN_FLOW"}, inplace=True)

NFD_MEAN_FLOW_DF.head()

,NFD_MEAN_FLOW
OBS DATE,
2008-01-01,152.0
2008-01-02,134.0
2008-01-03,126.0
2008-01-04,NaN
2008-01-05,NaN


In [137]:
### Auburn Dam Ridge (ADR) ###
# Elevation: 1,200 ft

# PRECIPITATION, ACCUMULATED, INCHES (daily)
ADR_PRECIP_ACC_DATA = pd.read_excel(f'{DATA_PATH}/CDEC_Water_Data/ADR/ADR_2.xlsx')
ADR_PRECIP_ACC_DF = prepare_station_df(ADR_PRECIP_ACC_DATA)

# PRECIPITATION, INCREMENTAL, INCHES (daily)
# Incremental precipitation = difference between current and preceding accumulated precipitation
ADR_PRECIP_INCR_DATA = pd.read_excel(f'{DATA_PATH}/CDEC_Water_Data/ADR/ADR_45.xlsx')
ADR_PRECIP_INCR_DF = prepare_station_df(ADR_PRECIP_INCR_DATA)

# TEMPERATURE, AIR AVERAGE, DEG F (daily)
ADR_TEMP_AVG_DATA = pd.read_excel(f'{DATA_PATH}/CDEC_Water_Data/ADR/ADR_30.xlsx')
ADR_TEMP_AVG_DF = prepare_station_df(ADR_TEMP_AVG_DATA)

# TEMPERATURE, AIR MAXIMUM, DEG F (daily)
ADR_TEMP_MAX_DATA = pd.read_excel(f'{DATA_PATH}/CDEC_Water_Data/ADR/ADR_31.xlsx')
ADR_TEMP_MAX_DF = prepare_station_df(ADR_TEMP_MAX_DATA)

# TEMPERATURE, AIR MINIMUM, DEG F (daily)
ADR_TEMP_MIN_DATA = pd.read_excel(f'{DATA_PATH}/CDEC_Water_Data/ADR/ADR_32.xlsx')
ADR_TEMP_MIN_DF = prepare_station_df(ADR_TEMP_MIN_DATA)


In [138]:
ADR_TEMP_MIN_DF.head()

,VALUE
OBS DATE,
2008-01-01,34.0
2008-01-02,39.0
2008-01-03,38.0
2008-01-04,42.0
2008-01-05,37.0


In [139]:
# Reset value name for df merge

ADR_PRECIP_ACC_DF.rename(columns={"VALUE": "ADR_PRECIP_ACC"}, inplace=True)
ADR_PRECIP_INCR_DF.rename(columns={"VALUE": "ADR_PRECIP_INCR"}, inplace=True)
ADR_TEMP_AVG_DF.rename(columns={"VALUE": "ADR_TEMP_AVG"}, inplace=True)
ADR_TEMP_MAX_DF.rename(columns={"VALUE": "ADR_TEMP_MAX"}, inplace=True)
ADR_TEMP_MIN_DF.rename(columns={"VALUE": "ADR_TEMP_MIN"}, inplace=True)

In [140]:
ADR_TEMP_MIN_DF.head()

,ADR_TEMP_MIN
OBS DATE,
2008-01-01,34.0
2008-01-02,39.0
2008-01-03,38.0
2008-01-04,42.0
2008-01-05,37.0


In [141]:
### Huysink (HYS) ###
# Elevation: 6,600 ft

# PRECIPITATION, ACCUMULATED, INCHES (daily)
HYS_PRECIP_ACC_DATA = pd.read_excel(f'{DATA_PATH}/CDEC_Water_Data/HYS/HYS_2.xlsx')
HYS_PRECIP_ACC_DF = prepare_station_df(HYS_PRECIP_ACC_DATA)

# PRECIPITATION, INCREMENTAL, INCHES (daily)
HYS_PRECIP_INCR_DATA = pd.read_excel(f'{DATA_PATH}/CDEC_Water_Data/HYS/HYS_45.xlsx')
HYS_PRECIP_INCR_DF = prepare_station_df(HYS_PRECIP_INCR_DATA)

# SNOW DEPTH, INCHES (daily)
HYS_SNOW_DEPTH_DATA = pd.read_excel(f'{DATA_PATH}/CDEC_Water_Data/HYS/HYS_18.xlsx')
HYS_SNOW_DEPTH_DF = prepare_station_df(HYS_SNOW_DEPTH_DATA)

# SNOW, WATER CONTENT, INCHES (daily)
HYS_SNOW_WATER_CONTENT_DATA = pd.read_excel(f'{DATA_PATH}/CDEC_Water_Data/HYS/HYS_3.xlsx')
HYS_SNOW_WATER_CONTENT_DF = prepare_station_df(HYS_SNOW_WATER_CONTENT_DATA)

# TEMPERATURE, AIR AVERAGE, DEG F (daily)
HYS_TEMP_AVG_DATA = pd.read_excel(f'{DATA_PATH}/CDEC_Water_Data/HYS/HYS_30.xlsx')
HYS_TEMP_AVG_DF = prepare_station_df(HYS_TEMP_AVG_DATA)

# TEMPERATURE, AIR MAXIMUM, DEG F (daily)
HYS_TEMP_MAX_DATA = pd.read_excel(f'{DATA_PATH}/CDEC_Water_Data/HYS/HYS_31.xlsx')
HYS_TEMP_MAX_DF = prepare_station_df(HYS_TEMP_MAX_DATA)

# TEMPERATURE, AIR MINIMUM, DEG F (daily)
HYS_TEMP_MIN_DATA = pd.read_excel(f'{DATA_PATH}/CDEC_Water_Data/HYS/HYS_32.xlsx')
HYS_TEMP_MIN_DF = prepare_station_df(HYS_TEMP_MIN_DATA)


In [142]:
# Reset value name for df merge

HYS_PRECIP_ACC_DF.rename(columns={"VALUE": "HYS_PRECIP_ACC"}, inplace=True)
HYS_PRECIP_INCR_DF.rename(columns={"VALUE": "HYS_PRECIP_INCR"}, inplace=True)
HYS_SNOW_DEPTH_DF.rename(columns={"VALUE": "HYS_SNOW_DEPTH"}, inplace=True)
HYS_SNOW_WATER_CONTENT_DF.rename(columns={"VALUE": "HYS_SNOW_WATER_CONTENT"}, inplace=True)
HYS_TEMP_AVG_DF.rename(columns={"VALUE": "HYS_TEMP_AVG"}, inplace=True)
HYS_TEMP_MAX_DF.rename(columns={"VALUE": "HYS_TEMP_MAX"}, inplace=True)
HYS_TEMP_MIN_DF.rename(columns={"VALUE": "HYS_TEMP_MIN"}, inplace=True)

In [143]:
HYS_SNOW_DEPTH_DF.info

<bound method DataFrame.info of             HYS_SNOW_DEPTH
OBS DATE                  
2008-01-01            38.0
2008-01-02            38.0
2008-01-03            39.0
2008-01-04            45.0
2008-01-05            58.0
...                    ...
2022-12-27            40.0
2022-12-28            41.0
2022-12-29            41.0
2022-12-30            43.0
2022-12-31            35.0

[5479 rows x 1 columns]>

In [144]:
HYS_SNOW_DEPTH_DF.tail()

,HYS_SNOW_DEPTH
OBS DATE,
2022-12-27,40.0
2022-12-28,41.0
2022-12-29,41.0
2022-12-30,43.0
2022-12-31,35.0


#### Near American River, Middle Fork

In [167]:
### MIDDLE FK AMERICAN R NR OXBOW PH (OXB) ###
# Elevation: 1,070 ft

# RIVER STAGE, FEET (hourly)
OXB_RIVER_STAGE_DATA = pd.read_excel(f'{DATA_PATH}/CDEC_Water_Data/OXB/OXB_1.xlsx')
OXB_RIVER_STAGE_DF = prepare_station_df(OXB_RIVER_STAGE_DATA, hourly=True, date_col='DATE TIME')

# Reset value name for df merge
OXB_RIVER_STAGE_DF.rename(columns={"VALUE": "OXB_RIVER_STAGE"}, inplace=True)

OXB_RIVER_STAGE_DF.head()

,OXB_RIVER_STAGE
DATE TIME,
2008-01-01,8.585417
2008-01-02,8.487917
2008-01-03,8.343333
2008-01-04,9.907083
2008-01-05,10.882500


In [168]:
# FLOW, RIVER DISCHARGE, CFS (hourly)
OXB_RIVER_DISCHARGE_DATA = pd.read_excel(f'{DATA_PATH}/CDEC_Water_Data/OXB/OXB_20.xlsx')

In [169]:
OXB_RIVER_DISCHARGE_DF = prepare_station_df(OXB_RIVER_DISCHARGE_DATA, hourly=True, date_col='DATE TIME')

# Reset value name for df merge
OXB_RIVER_DISCHARGE_DF.rename(columns={"VALUE": "OXB_RIVER_DISCHARGE"}, inplace=True)

In [170]:
OXB_RIVER_DISCHARGE_DF.head()

,OXB_RIVER_DISCHARGE
DATE TIME,
2008-01-01,169.666667
2008-01-02,153.958333
2008-01-03,132.750000
2008-01-04,752.833333
2008-01-05,927.250000


In [151]:
### Duncan (DUN) ###
# Elevation: 7,100 ft

# PRECIPITATION, ACCUMULATED, INCHES (hourly)
DUN_PRECIP_ACC_DATA = pd.read_excel(f'{DATA_PATH}/CDEC_Water_Data/DUN/DUN_2.xlsx')
DUN_PRECIP_ACC_DF = prepare_station_df(DUN_PRECIP_ACC_DATA, hourly=True)

# TEMPERATURE, AIR AVERAGE, DEG F (daily)
DUN_TEMP_AVG_DATA = pd.read_excel(f'{DATA_PATH}/CDEC_Water_Data/DUN/DUN_30.xlsx')
DUN_TEMP_AVG_DF = prepare_station_df(DUN_TEMP_AVG_DATA)

# TEMPERATURE, AIR MAXIMUM, DEG F (daily)
DUN_TEMP_MAX_DATA = pd.read_excel(f'{DATA_PATH}/CDEC_Water_Data/DUN/DUN_31.xlsx')
DUN_TEMP_MAX_DF = prepare_station_df(DUN_TEMP_MAX_DATA)

# TEMPERATURE, AIR MINIMUM, DEG F (daily)
DUN_TEMP_MIN_DATA = pd.read_excel(f'{DATA_PATH}/CDEC_Water_Data/DUN/DUN_32.xlsx')
DUN_TEMP_MIN_DF = prepare_station_df(DUN_TEMP_MIN_DATA)


In [152]:
# Reset value name for df merge

DUN_PRECIP_ACC_DF.rename(columns={"VALUE": "DUN_PRECIP_ACC"}, inplace=True)
DUN_TEMP_AVG_DF.rename(columns={"VALUE": "DUN_TEMP_AVG"}, inplace=True)
DUN_TEMP_MAX_DF.rename(columns={"VALUE": "DUN_TEMP_MAX"}, inplace=True)
DUN_TEMP_MIN_DF.rename(columns={"VALUE": "DUN_TEMP_MIN"}, inplace=True)

In [153]:
DUN_TEMP_MIN_DF.head()

,DUN_TEMP_MIN
OBS DATE,
2008-01-01,39.0
2008-01-02,32.0
2008-01-03,26.0
2008-01-04,NaN
2008-01-05,NaN


In [154]:
### Sugar Pine (SGP) ###
# Elevation: 3,843 ft

# PRECIPITATION, ACCUMULATED, INCHES (daily)
SGP_PRECIP_ACC_DATA = pd.read_excel(f'{DATA_PATH}/CDEC_Water_Data/SGP/SGP_2.xlsx')
SGP_PRECIP_ACC_DF = prepare_station_df(SGP_PRECIP_ACC_DATA)

# PRECIPITATION, INCREMENTAL, INCHES (daily)
SGP_PRECIP_INCR_DATA = pd.read_excel(f'{DATA_PATH}/CDEC_Water_Data/SGP/SGP_45.xlsx')
SGP_PRECIP_INCR_DF = prepare_station_df(SGP_PRECIP_INCR_DATA)

# TEMPERATURE, AIR AVERAGE, DEG F (daily)
SGP_TEMP_AVG_DATA = pd.read_excel(f'{DATA_PATH}/CDEC_Water_Data/SGP/SGP_30.xlsx')
SGP_TEMP_AVG_DF = prepare_station_df(SGP_TEMP_AVG_DATA)

# TEMPERATURE, AIR MAXIMUM, DEG F (daily)
SGP_TEMP_MAX_DATA = pd.read_excel(f'{DATA_PATH}/CDEC_Water_Data/SGP/SGP_31.xlsx')
SGP_TEMP_MAX_DF = prepare_station_df(SGP_TEMP_MAX_DATA)

# TEMPERATURE, AIR MINIMUM, DEG F (daily)
SGP_TEMP_MIN_DATA = pd.read_excel(f'{DATA_PATH}/CDEC_Water_Data/SGP/SGP_32.xlsx')
SGP_TEMP_MIN_DF = prepare_station_df(SGP_TEMP_MIN_DATA)


In [155]:
# Reset value name for df merge

SGP_PRECIP_ACC_DF.rename(columns={"VALUE": "SGP_PRECIP_ACC"}, inplace=True)
SGP_PRECIP_INCR_DF.rename(columns={"VALUE": "SGP_PRECIP_INCR"}, inplace=True)
SGP_TEMP_AVG_DF.rename(columns={"VALUE": "SGP_TEMP_AVG"}, inplace=True)
SGP_TEMP_MAX_DF.rename(columns={"VALUE": "SGP_TEMP_MAX"}, inplace=True)
SGP_TEMP_MIN_DF.rename(columns={"VALUE": "SGP_TEMP_MIN"}, inplace=True)

In [156]:
SGP_TEMP_MIN_DF.head()

,SGP_TEMP_MIN
OBS DATE,
2008-01-01,27.0
2008-01-02,27.0
2008-01-03,34.0
2008-01-04,32.0
2008-01-05,31.0


#### Near American River, South Fork

In [171]:
### AMERICAN RIVER AT CHILI BAR (CBR) ###
# Elevation: 931 ft

# RIVER STAGE, FEET (hourly)
CBR_RIVER_STAGE_DATA = pd.read_excel(f'{DATA_PATH}/CDEC_Water_Data/CBR/CBR_1.xlsx')
CBR_RIVER_STAGE_DF = prepare_station_df(CBR_RIVER_STAGE_DATA, hourly=True)

# Reset value name for df merge
CBR_RIVER_STAGE_DF.rename(columns={"VALUE": "CBR_RIVER_STAGE"}, inplace=True)

CBR_RIVER_STAGE_DF.head()

,CBR_RIVER_STAGE
OBS DATE,
2008-01-01,4.445833
2008-01-02,2.298750
2008-01-03,2.993333
2008-01-04,2.649583
2008-01-05,3.432083


In [172]:
# FLOW, RIVER DISCHARGE, CFS (hourly)
CBR_RIVER_DISCHARGE_DATA = pd.read_excel(f'{DATA_PATH}/CDEC_Water_Data/CBR/CBR_20.xlsx')

In [173]:
CBR_RIVER_DISCHARGE_DF = prepare_station_df(CBR_RIVER_DISCHARGE_DATA, hourly=True)

# Reset value name for df merge
CBR_RIVER_DISCHARGE_DF.rename(columns={"VALUE": "CBR_RIVER_DISCHARGE"}, inplace=True)

CBR_RIVER_DISCHARGE_DF.head()

,CBR_RIVER_DISCHARGE
OBS DATE,
2008-01-01,281.045455
2008-01-02,722.583333
2008-01-03,1115.916667
2008-01-04,866.958333
2008-01-05,1273.791667


In [179]:
### Forni Ridge (FRN) ###
# Elevation: 7,600 ft

# PRECIPITATION, ACCUMULATED, INCHES (daily)
FRN_PRECIP_ACC_DATA = pd.read_excel(f'{DATA_PATH}/CDEC_Water_Data/FRN/FRN_2.xlsx')
FRN_PRECIP_ACC_DF = prepare_station_df(FRN_PRECIP_ACC_DATA)

# PRECIPITATION, INCREMENTAL, INCHES (daily)
FRN_PRECIP_INCR_DATA = pd.read_excel(f'{DATA_PATH}/CDEC_Water_Data/FRN/FRN_45.xlsx')
FRN_PRECIP_INCR_DF = prepare_station_df(FRN_PRECIP_INCR_DATA)

# SNOW DEPTH, INCHES (daily)
FRN_SNOW_DEPTH_DATA = pd.read_excel(f'{DATA_PATH}/CDEC_Water_Data/FRN/FRN_18.xlsx')
FRN_SNOW_DEPTH_DF = prepare_station_df(FRN_SNOW_DEPTH_DATA)

# SNOW, WATER CONTENT, INCHES (daily)
FRN_SNOW_WATER_CONTENT_DATA = pd.read_excel(f'{DATA_PATH}/CDEC_Water_Data/FRN/FRN_3.xlsx')
FRN_SNOW_WATER_CONTENT_DF = prepare_station_df(FRN_SNOW_WATER_CONTENT_DATA)

# TEMPERATURE, AIR AVERAGE, DEG F (daily)
FRN_TEMP_AVG_DATA = pd.read_excel(f'{DATA_PATH}/CDEC_Water_Data/FRN/FRN_30.xlsx')
FRN_TEMP_AVG_DF = prepare_station_df(FRN_TEMP_AVG_DATA)

# TEMPERATURE, AIR MAXIMUM, DEG F (daily)
FRN_TEMP_MAX_DATA = pd.read_excel(f'{DATA_PATH}/CDEC_Water_Data/FRN/FRN_31.xlsx')
FRN_TEMP_MAX_DF = prepare_station_df(FRN_TEMP_MAX_DATA)

# TEMPERATURE, AIR MINIMUM, DEG F (daily)
FRN_TEMP_MIN_DATA = pd.read_excel(f'{DATA_PATH}/CDEC_Water_Data/FRN/FRN_32.xlsx')
FRN_TEMP_MIN_DF = prepare_station_df(FRN_TEMP_MIN_DATA)


In [180]:
# Reset value name for df merge

FRN_PRECIP_ACC_DF.rename(columns={"VALUE": "FRN_PRECIP_ACC"}, inplace=True)
FRN_PRECIP_INCR_DF.rename(columns={"VALUE": "FRN_PRECIP_INCR"}, inplace=True)
FRN_SNOW_DEPTH_DF.rename(columns={"VALUE": "FRN_SNOW_DEPTH"}, inplace=True)
FRN_SNOW_WATER_CONTENT_DF.rename(columns={"VALUE": "FRN_SNOW_WATER_CONTENT"}, inplace=True)
FRN_TEMP_AVG_DF.rename(columns={"VALUE": "FRN_TEMP_AVG"}, inplace=True)
FRN_TEMP_MAX_DF.rename(columns={"VALUE": "FRN_TEMP_MAX"}, inplace=True)
FRN_TEMP_MIN_DF.rename(columns={"VALUE": "FRN_TEMP_MIN"}, inplace=True)

In [181]:
FRN_TEMP_MIN_DF.head()

,FRN_TEMP_MIN
OBS DATE,
2008-01-01,37.0
2008-01-02,32.0
2008-01-03,25.0
2008-01-04,28.0
2008-01-05,NaN


In [182]:
### Pacific House (PFH) ###
# Elevation: 3,440 ft

# PRECIPITATION, ACCUMULATED, INCHES (daily)
PFH_PRECIP_ACC_DATA = pd.read_excel(f'{DATA_PATH}/CDEC_Water_Data/PFH/PFH_2.xlsx')
PFH_PRECIP_ACC_DF = prepare_station_df(PFH_PRECIP_ACC_DATA)

# PRECIPITATION, INCREMENTAL, INCHES (daily)
PFH_PRECIP_INCR_DATA = pd.read_excel(f'{DATA_PATH}/CDEC_Water_Data/PFH/PFH_45.xlsx')
PFH_PRECIP_INCR_DF = prepare_station_df(PFH_PRECIP_INCR_DATA)

# TEMPERATURE, AIR AVERAGE, DEG F (daily)
PFH_TEMP_AVG_DATA = pd.read_excel(f'{DATA_PATH}/CDEC_Water_Data/PFH/PFH_30.xlsx')
PFH_TEMP_AVG_DF = prepare_station_df(PFH_TEMP_AVG_DATA)

# TEMPERATURE, AIR MAXIMUM, DEG F (daily)
PFH_TEMP_MAX_DATA = pd.read_excel(f'{DATA_PATH}/CDEC_Water_Data/PFH/PFH_31.xlsx')
PFH_TEMP_MAX_DF = prepare_station_df(PFH_TEMP_MAX_DATA)

# TEMPERATURE, AIR MINIMUM, DEG F (daily)
PFH_TEMP_MIN_DATA = pd.read_excel(f'{DATA_PATH}/CDEC_Water_Data/PFH/PFH_32.xlsx')
PFH_TEMP_MIN_DF = prepare_station_df(PFH_TEMP_MIN_DATA)


In [183]:
# Reset value name for df merge

PFH_PRECIP_ACC_DF.rename(columns={"VALUE": "PFH_PRECIP_ACC"}, inplace=True)
PFH_PRECIP_INCR_DF.rename(columns={"VALUE": "PFH_PRECIP_INCR"}, inplace=True)
PFH_TEMP_AVG_DF.rename(columns={"VALUE": "PFH_TEMP_AVG"}, inplace=True)
PFH_TEMP_MAX_DF.rename(columns={"VALUE": "PFH_TEMP_MAX"}, inplace=True)
PFH_TEMP_MIN_DF.rename(columns={"VALUE": "PFH_TEMP_MIN"}, inplace=True)

In [184]:
PFH_TEMP_MIN_DF.head()

,PFH_TEMP_MIN
OBS DATE,
2008-01-01,30.0
2008-01-02,29.0
2008-01-03,29.0
2008-01-04,32.0
2008-01-05,32.0


In [185]:
# Create Reservoir Basin Inflow dataset, merging CDEC data

STATION_DF = [NFD_MEAN_FLOW_DF, OXB_RIVER_STAGE_DF, OXB_RIVER_DISCHARGE_DF, CBR_RIVER_STAGE_DF, CBR_RIVER_DISCHARGE_DF, ADR_PRECIP_ACC_DF, ADR_PRECIP_INCR_DF, ADR_TEMP_AVG_DF, ADR_TEMP_MAX_DF, ADR_TEMP_MIN_DF, HYS_PRECIP_ACC_DF, HYS_PRECIP_INCR_DF, HYS_SNOW_DEPTH_DF, HYS_SNOW_WATER_CONTENT_DF, HYS_TEMP_AVG_DF, HYS_TEMP_MAX_DF, HYS_TEMP_MIN_DF, DUN_PRECIP_ACC_DF, DUN_TEMP_AVG_DF, DUN_TEMP_MAX_DF, DUN_TEMP_MIN_DF, SGP_PRECIP_ACC_DF, SGP_PRECIP_INCR_DF, SGP_TEMP_AVG_DF, SGP_TEMP_MAX_DF, SGP_TEMP_MIN_DF, FRN_PRECIP_ACC_DF, FRN_PRECIP_INCR_DF, FRN_SNOW_DEPTH_DF, FRN_SNOW_WATER_CONTENT_DF, FRN_TEMP_AVG_DF, FRN_TEMP_MAX_DF, FRN_TEMP_MIN_DF, PFH_PRECIP_ACC_DF, PFH_PRECIP_INCR_DF, PFH_TEMP_AVG_DF, PFH_TEMP_MAX_DF, PFH_TEMP_MIN_DF]

RES_BASIN_INFLOW = FOL_INFLOW_DATA_DF.join(STATION_DF)

In [186]:
RES_BASIN_INFLOW.columns

Index(['INFLOW', 'NFD_MEAN_FLOW', 'OXB_RIVER_STAGE', 'OXB_RIVER_DISCHARGE',
       'CBR_RIVER_STAGE', 'CBR_RIVER_DISCHARGE', 'ADR_PRECIP_ACC',
       'ADR_PRECIP_INCR', 'ADR_TEMP_AVG', 'ADR_TEMP_MAX', 'ADR_TEMP_MIN',
       'HYS_PRECIP_ACC', 'HYS_PRECIP_INCR', 'HYS_SNOW_DEPTH',
       'HYS_SNOW_WATER_CONTENT', 'HYS_TEMP_AVG', 'HYS_TEMP_MAX',
       'HYS_TEMP_MIN', 'DUN_PRECIP_ACC', 'DUN_TEMP_AVG', 'DUN_TEMP_MAX',
       'DUN_TEMP_MIN', 'SGP_PRECIP_ACC', 'SGP_PRECIP_INCR', 'SGP_TEMP_AVG',
       'SGP_TEMP_MAX', 'SGP_TEMP_MIN', 'FRN_PRECIP_ACC', 'FRN_PRECIP_INCR',
       'FRN_SNOW_DEPTH', 'FRN_SNOW_WATER_CONTENT', 'FRN_TEMP_AVG',
       'FRN_TEMP_MAX', 'FRN_TEMP_MIN', 'PFH_PRECIP_ACC', 'PFH_PRECIP_INCR',
       'PFH_TEMP_AVG', 'PFH_TEMP_MAX', 'PFH_TEMP_MIN'],
      dtype='object')

In [188]:
RES_BASIN_INFLOW.head()

,INFLOW,NFD_MEAN_FLOW,OXB_RIVER_STAGE,OXB_RIVER_DISCHARGE,CBR_RIVER_STAGE,CBR_RIVER_DISCHARGE,ADR_PRECIP_ACC,ADR_PRECIP_INCR,ADR_TEMP_AVG,ADR_TEMP_MAX,...,FRN_SNOW_DEPTH,FRN_SNOW_WATER_CONTENT,FRN_TEMP_AVG,FRN_TEMP_MAX,FRN_TEMP_MIN,PFH_PRECIP_ACC,PFH_PRECIP_INCR,PFH_TEMP_AVG,PFH_TEMP_MAX,PFH_TEMP_MIN
OBS DATE,,,,,,,,,,,,,,,,,,,,,
2008-01-01,977.0,152.0,8.585417,169.666667,4.445833,281.045455,9.52,0.00,44.0,58.0,...,30.0,7.90,44.0,55.0,37.0,7.72,0.04,40.0,61.0,30.0
2008-01-02,1061.0,134.0,8.487917,153.958333,2.298750,722.583333,9.52,0.00,47.0,57.0,...,30.0,7.88,38.0,46.0,32.0,7.76,0.04,39.0,63.0,29.0
2008-01-03,1712.0,126.0,8.343333,132.750000,2.993333,1115.916667,9.76,0.24,45.0,50.0,...,30.0,7.86,30.0,35.0,25.0,7.76,0.00,40.0,53.0,29.0
2008-01-04,NaN,NaN,9.907083,752.833333,2.649583,866.958333,11.28,1.52,46.0,50.0,...,33.0,8.27,31.0,33.0,28.0,11.00,3.24,39.0,47.0,32.0
2008-01-05,7072.0,NaN,10.882500,927.250000,3.432083,1273.791667,11.96,0.68,41.0,46.0,...,NaN,NaN,NaN,NaN,NaN,12.04,1.04,33.0,38.0,32.0


In [187]:
RES_BASIN_INFLOW.info

<bound method DataFrame.info of               INFLOW  NFD_MEAN_FLOW  OXB_RIVER_STAGE  OXB_RIVER_DISCHARGE  \
OBS DATE                                                                    
2008-01-01     977.0          152.0         8.585417           169.666667   
2008-01-02    1061.0          134.0         8.487917           153.958333   
2008-01-03    1712.0          126.0         8.343333           132.750000   
2008-01-04       NaN            NaN         9.907083           752.833333   
2008-01-05    7072.0            NaN        10.882500           927.250000   
...              ...            ...              ...                  ...   
2022-12-27    7838.0         3190.0        11.834167          2894.500000   
2022-12-28    7630.0         2569.0        11.619167          2229.875000   
2022-12-29    7172.0         1208.0        10.390870          1320.869565   
2022-12-30   21529.0         8055.0        11.987826          2867.434783   
2022-12-31  106510.0        20749.0        1

In [189]:
print("Null/NaN count: ", RES_BASIN_INFLOW.isnull().sum().sum())

Null/NaN count:  15492


In [190]:
def handle_missing(df):
  # forward fill missing values
  df.ffill(axis=0, inplace=True)

In [191]:
handle_missing(RES_BASIN_INFLOW)
print("Null/NaN count: ", RES_BASIN_INFLOW.isnull().sum().sum())

Null/NaN count:  0


In [193]:
RES_BASIN_INFLOW.dtypes

INFLOW                    float64
NFD_MEAN_FLOW             float64
OXB_RIVER_STAGE           float64
OXB_RIVER_DISCHARGE       float64
CBR_RIVER_STAGE           float64
CBR_RIVER_DISCHARGE       float64
ADR_PRECIP_ACC            float64
ADR_PRECIP_INCR           float64
ADR_TEMP_AVG              float64
ADR_TEMP_MAX              float64
ADR_TEMP_MIN              float64
HYS_PRECIP_ACC            float64
HYS_PRECIP_INCR           float64
HYS_SNOW_DEPTH            float64
HYS_SNOW_WATER_CONTENT    float64
HYS_TEMP_AVG              float64
HYS_TEMP_MAX              float64
HYS_TEMP_MIN              float64
DUN_PRECIP_ACC            float64
DUN_TEMP_AVG              float64
DUN_TEMP_MAX              float64
DUN_TEMP_MIN              float64
SGP_PRECIP_ACC            float64
SGP_PRECIP_INCR           float64
SGP_TEMP_AVG              float64
SGP_TEMP_MAX              float64
SGP_TEMP_MIN              float64
FRN_PRECIP_ACC            float64
FRN_PRECIP_INCR           float64
FRN_SNOW_DEPTH

In [194]:
RES_BASIN_INFLOW.to_excel(f'{DATA_PATH}/Custom/basin_inflow.xlsx')